In [1]:
import gym
import tensorflow as tf
import numpy as np
import time
import random
from collections import deque
from IPython.display import clear_output

tf.compat.v1.reset_default_graph()
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 


epsilon = 1.0
epsilon_min = 0.01
decay_rate = 0.005
replay_memory = deque(maxlen=1000)
batch_size = 20
gamma = 0.99

S = tf.placeholder(tf.float32, [None, 4])
A = tf.placeholder(tf.float32, [None, 1])
target = tf.placeholder(tf.float32, [None, 1])
act_index = tf.placeholder(tf.int32, [None, 2])

# Online variables
W1s = tf.Variable(tf.random_normal([4, 10], stddev=0.1))
W1a = tf.Variable(tf.random_normal([1, 10], stddev=0.1))
b1 = tf.Variable(tf.zeros([10]))
L1 = tf.nn.relu(tf.matmul(S, W1s) + tf.matmul(A, W1a) + b1)
W2 = tf.Variable(tf.random_normal([10, 1], stddev=0.1))
Q_online = tf.matmul(L1, W2)

# Target variables
W1st = tf.Variable(tf.random_normal([4, 10], stddev=0.1))
W1at = tf.Variable(tf.random_normal([1, 10], stddev=0.1))
b1t = tf.Variable(tf.zeros([10]))
L1t = tf.nn.relu(tf.matmul(S, W1st) + tf.matmul(A, W1at) + b1t)
W2t = tf.Variable(tf.random_normal([10, 1], stddev=0.1))
Q_target = tf.matmul(L1t, W2t)


loss = tf.reduce_mean((target - Q_online) ** 2)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

W1st.load(sess.run(W1s),sess)
W1at.load(sess.run(W1a),sess)
b1t.load(sess.run(b1),sess)
W2t.load(sess.run(W2),sess)



env = gym.make("CartPole-v0")

counter =0
for episode in range(30000):
    obs = env.reset()
    obs = obs.reshape(-1, 4)
    done = False
    score = 0
    while not done:
        counter = counter + 1
        if random.random() < epsilon:
            action = random.randint(0, 1)
        else:
           
            Q1 = sess.run([Q_online], feed_dict={S: obs, A: [[0]]})
            Q2 = sess.run([Q_online], feed_dict={S: obs, A: [[1]]})
            action = np.argmax(np.array([Q1, Q2]))

        
        # do action and get new state, reward and done
        new_obs, reward, done, _ = env.step(action)
        new_obs = new_obs.reshape(-1, 4)
        score = score + reward

        if done == True:
            terminal = 1
        else: 
            terminal = 0
        transition = [obs[0], [action], reward, new_obs[0], terminal]
        replay_memory.append(transition)
        
        obs = new_obs
        
        
        if len(replay_memory) >= batch_size:
            train_data = random.sample(replay_memory, batch_size)
            state   = [data[0] for data in train_data]
            action      = [data[1] for data in train_data]
            reward      = np.array([data[2] for data in train_data])
            new_state   = [data[3] for data in train_data]
            terminal    = np.array([data[4] for data in train_data])
            
            state = np.stack(state)
            new_state = np.stack(new_state)

            
            q_next1 = sess.run([Q_target], feed_dict={S: new_state, A: [[0]]})
            q_next2 = sess.run([Q_target], feed_dict={S: new_state, A: [[1]]})

            # set batch target value : r + gamma * max(q-value)
            batch_target = reward + gamma * np.max([q_next1, q_next2]) * (1-terminal)
            batch_target = batch_target.reshape(-1, 1)
            
            loss_var, _ = sess.run([loss, optimizer], feed_dict={S: state, A: action, target: batch_target})
            
            if counter % 5000 == 0:
                # Target variable update (soft update)
                W1st.load(0.5*sess.run(W1s) + 0.5*sess.run(W1st),sess)
                W1at.load(0.5*sess.run(W1a) + 0.5*sess.run(W1at),sess)
                b1t.load(0.5*sess.run(b1) + 0.5*sess.run(b1t),sess)
                W2t.load(0.5*sess.run(W2) + 0.5*sess.run(W2t),sess)
                print('Target updated')
                
            if episode % 100 == 0:
                clear_output(wait=True)
                print('Episode: {} Score: {}'.format(episode, score))

            if epsilon > epsilon_min:
                epsilon = epsilon - decay_rate


# TEST     
episode = 0
state = env.reset()     
while episode < 10:  # episode loop
    #env.render()
    Q1 = sess.run([Q_online], feed_dict={S: obs, A: [[0]]})
    Q2 = sess.run([Q_online], feed_dict={S: obs, A: [[1]]})
    action = np.argmax(np.array([Q1, Q2]))
    
    next_state, reward, done, info = env.step(action)  # take a random action
    state = next_state

    if done:
        episode = episode + 1
        state = env.reset()
env.close()    


Episode: 29900 Score: 8.0
Target updated
